### Dataset taken is kaggle IMDB-5000-movie-dataset

### https://www.kaggle.com/deepmatrix/imdb-5000-movie-dataset

With 28 variables-- 
"movie_title" "color" "num_critic_for_reviews" "movie_facebook_likes" "duration" "director_name" "director_facebook_likes" "actor_3_name" "actor_3_facebook_likes" "actor_2_name" "actor_2_facebook_likes" "actor_1_name" "actor_1_facebook_likes" "gross" "genres" "num_voted_users" "cast_total_facebook_likes" "facenumber_in_poster" "plot_keywords" "movie_imdb_link" "num_user_for_reviews" "language" "country" "content_rating" "budget" "title_year" "imdb_score" "aspect_ratio"


### Install the pyorient package!

In [2]:
######## download pyorientdb 
! pip install pyorient --user 

### Import all modules required for the script

In [2]:
import pyorient, json, pandas as pd
import sys



##### If you are using the Data Science Experience ( DSX), set up the orientdb on kubernetes. Follow the developer journey https://github.com/IBM/deploy-graph-db-container. This is because if you set up the orientdb locally, you won’t able to access it through DSX.As the orientdb console port 2424 and orientdb studio 2480  wouldn’t be exposed on bluemix. Setting up the orientdb on kubernetes will expose the ports on bluemix through which orientdb can be accessed from the data science experience.You can use the ip-address of your cluster and node port on which the port 2424 orientdb console is mapped, to access that orientdb through notebook.


### Add your service credentials for Object Storage
You must create Object Storage service on Bluemix. To access data in a file in Object Storage, you need the Object Storage authentication credentials. Click on insert to code for the file you want to use in the code.

In [4]:
# # @hidden_cell
credentials_1 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_3fd48083_3bbd_4355_a296_f2558dc1b897',
  'project_id':'07704599ed444df0927832e455d9ec63',
  'region':'dallas',
  'user_id':'fb16c3d0955c42538faa723269123f5d',
  'domain_id':'1bfd0aa070764f39845124cf77d75a19',
  'domain_name':'1104675',
  'username':'member_b6c199f708035093454e23f2785a418ef096423b',
  'password':"""VO~S.?)5E6g2!!Dh""",
  'container':'graphdb',
  'tenantId':'undefined',
  'filename':'config.json'
}


In [8]:
from io import BytesIO  
import requests  
import json  
import pandas as pd

def get_data(credentials):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', credentials['filename']])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return json.loads(resp2.content)
# This code returns a Python dictionary with your JSON data. If you want this data in a Pandas data frame, use the following code:


In [6]:
json_data = get_data(credentials_1) 

## Load the config file 

In [7]:
node_data = json.dumps(json_data)
node_data = json.loads(node_data)
print(node_data)
type(node_data)

{u'username': u'root', u'edge_class': {u'acted_in': {u'in': {u'Linked_Class': u'movie', u'Type': u'Link'}, u'out': {u'Linked_Class': u'person', u'Type': u'Link'}}, u'worked_with': {u'in': {u'Linked_Class': u'person', u'Type': u'Link'}, u'out': {u'Linked_Class': u'person', u'Type': u'Link'}}}, u'Database_name': u'testdb', u'vertex_class': {u'person': {u'role': u'String', u'name': u'String', u'fblikes': u'Float'}, u'movie': {u'title': u'String', u'numCriticForReviews': u'Integer', u'plotKeywords': u'String', u'durationInMins': u'Integer', u'year': u'Integer', u'genre': u'String', u'movieFacebookLikes': u'Float', u'imdbRating': u'Float'}}, u'host': u'localhost', u'password': u'root', u'port': 2424}


dict

In [9]:
node_data['password'] = "rootpwd"
print node_data['username'], node_data['password']

root rootpwd


##  Connecting to Orientdb

In [2]:
## Please use the 2424/node port for 2424 to access orientdb. 
# The other port 2480/nodeport for 2480 is for orientdb studio.
# replace the content in the angular brackets.
client = pyorient.OrientDB(<ip-address-of-the-kubernetes-cluster>,<node-port mapped to port 2424 of orientdb>)

print(client)


### session-id username and password is global paswword, you have set for orientdb
session_id = client.connect(node_data['username'], node_data['password'])





### Creating a new database!

Orientdb provides an interactive studio to create new graph database, create schema, & add, delete, update records. You can follow the documentation(readme file) of this project which give a comprehensive tutorial on how to use orientdb console. 

You can follow this script to access orientdb through python.


In [11]:
#### Method for creating Database ################
def createDatabase(node_data):
    # check if it exists or not
    if client.db_exists( node_data['Database_name'], pyorient.STORAGE_TYPE_MEMORY ):
        client.db_open(node_data['Database_name'],node_data['username'], node_data['password'])
        print node_data['Database_name'] + " "+ "has already been created and opened for use"
    else:
    # create the database
        client.db_create( node_data['Database_name'], pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY )
        print node_data['Database_name'] + " created and opened successfully"
     

In [12]:
createDatabase(node_data)

testdb has already been created and opened for use


### Creating the graph.

### Node( vertex class)
There will be two vertex classes namely - Person and Movie.
#### Person’s attribute
Role: Director/Actor
Name
Fb-likes
#### Movie’s Attribute:
Title
Year
IMDB rating
Duration
Language
Genre
Plot keywords
Num_critic_for_reviews
movie_facebook_likes


In [17]:
######## method to check if class is present or not 
def check_if_class_present_or_not(classname):
    name="will be replaced by query result name"
    query = "SELECT FROM ( SELECT expand( classes ) FROM metadata:schema ) WHERE name = " +'"'+ classname + '"'
    print(query)
    a = client.command(query)
    
    for k in a:
        name = k.name
    
    if(name == classname):
        return True
    else:
        return False

In [18]:
#################### method for creating all the node classes and its properties #################################

############# use this function when you have designed schema ######################################
def createNodeClass_withSchema(node_data):
    
    for class_name,value in node_data['vertex_class'].items():
        bool_result = check_if_class_present_or_not(class_name)
        print bool_result
        if(not bool_result):
            command_to_create_node = "create class"+" "+ class_name +" "+ "extends V"
            print(command_to_create_node)
            cluster_id = client.command(command_to_create_node) 
            print class_name,cluster_id
            for property_name,value in node_data['vertex_class'][class_name].items():
                ### create properties 
                command_to_create_property= "create property"+ " "+ class_name +"." + property_name +" " +"IF NOT EXISTS " + value
                print(command_to_create_property)
                client.command(command_to_create_property) 
            print "class " + class_name + " and its properties successfully created"
        else:
            print "this class is already present !!"


############################### use this method to create class when you dont have deisgned/ fixed schema####################

def createNodeClass_NoSchema(node_data):
    for class_name,value in node_data['vertex_class'].items():
        if(check_if_class_present_or_not(class_name)):
            command_to_create_node = "create class"+" "+ class_name +" "+ "extends V"
            print(command_to_create_node)
            cluster_id = client.command(command_to_create_node) 
            print class_name + " has been created with cluster id " +cluster_id
        else:
            print "this class is already present !!"

In [ ]:
#################### uncomment the function you want to use ###################################
createNodeClass_withSchema(node_data)
#createNodeClass_NoSchema(node_data)

SELECT FROM ( SELECT expand( classes ) FROM metadata:schema ) WHERE name = "person"


#### Relations (Edge classes)
acted_in
worked_with

In Graph databases, a relation is the connection between two nodes, which in OrientDB is called an edge. Edges are bidirectional and can only connect two vertices. With the Graph API, Edges connect only two vertices.This means that 1:n relationships are not allowed. To specify a 1:n relationship with graphs, create multiple edges.

command to create the class:
create class Email_ex extends E

In [ ]:
# method for creating edge/link between the two nodes when you have designed schema for your graphdb
def createEdgeClass(node_data):
    for class_name,v in node_data['edge_class'].items():
        if(not check_if_class_present_or_not(class_name)):
            command_to_create_edge_class = "create class"+" "+ class_name +" "+ "extends E"
            print(command_to_create_edge_class)
            cluster_id = client.command(command_to_create_edge_class)
            print("Edge class" +" " + class_name + "has been created successfully!")

            for key,val in node_data['edge_class'][class_name].items():
                print key,val
                command_to_create_property= "create property"+ " "+ class_name +"." + key +" " +"IF NOT EXISTS " + node_data['edge_class'][class_name][key]['Type']+" " + node_data['edge_class'][class_name][key]['Linked_Class']
                print(command_to_create_property)
                client.command(command_to_create_property) 
                print node_data['edge_class'][class_name][key]['Type'], node_data['edge_class'][class_name][key]['Linked_Class']
        else:
             print "this class is already present !!"
            



In [ ]:
createEdgeClass(node_data)

In [27]:
######### loading the imdb movie data ##################################
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_3fd480833bbd4355a296f2558dc1b897(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_b6c199f708035093454e23f2785a418ef096423b','domain': {'id': '1bfd0aa070764f39845124cf77d75a19'},
            'password': 'VO~S.?)5E6g2!!Dh'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

imdb_df = pd.read_csv(get_object_storage_file_with_credentials_3fd480833bbd4355a296f2558dc1b897('graphdb', 'INSIGHTS-GRAPHDB.csv'))
imdb_df.head()


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723,178,0,855,Joel David Moore,1000,760505847,Action|Adventure|Fantasy|Sci-Fi,...,3054,English,USA,PG-13,237000000,2009,936,7.9,1.78,33000
1,Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,309404152,Action|Adventure|Fantasy,...,1238,English,USA,PG-13,300000000,2007,5000,7.1,2.35,0
2,Color,Sam Mendes,602,148,0,161,Rory Kinnear,11000,200074175,Action|Adventure|Thriller,...,994,English,UK,PG-13,245000000,2015,393,6.8,2.35,85000
3,Color,Christopher Nolan,813,164,22000,23000,Christian Bale,27000,448130642,Action|Thriller,...,2701,English,USA,PG-13,250000000,2012,23000,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131,NaN,Rob Walker,131,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12,7.1,NaN,0


In [28]:
################ dropping incomplete records ##############
imdb_df = imdb_df.dropna()
imdb_df

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723,178,0,855,Joel David Moore,1000,760505847,Action|Adventure|Fantasy|Sci-Fi,...,3054,English,USA,PG-13,237000000,2009,936,7.9,1.78,33000
1,Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,309404152,Action|Adventure|Fantasy,...,1238,English,USA,PG-13,300000000,2007,5000,7.1,2.35,0
2,Color,Sam Mendes,602,148,0,161,Rory Kinnear,11000,200074175,Action|Adventure|Thriller,...,994,English,UK,PG-13,245000000,2015,393,6.8,2.35,85000
3,Color,Christopher Nolan,813,164,22000,23000,Christian Bale,27000,448130642,Action|Thriller,...,2701,English,USA,PG-13,250000000,2012,23000,8.5,2.35,164000
5,Color,Andrew Stanton,462,132,475,530,Samantha Morton,640,73058679,Action|Adventure|Sci-Fi,...,738,English,USA,PG-13,263700000,2012,632,6.6,2.35,24000
6,Color,Sam Raimi,392,156,0,4000,James Franco,24000,336530303,Action|Adventure|Romance,...,1902,English,USA,PG-13,258000000,2007,11000,6.2,2.35,0
7,Color,Nathan Greno,324,100,15,284,Donna Murphy,799,200807262,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,387,English,USA,PG,260000000,2010,553,7.8,1.85,29000
8,Color,Joss Whedon,635,141,0,19000,Robert Downey Jr.,26000,458991599,Action|Adventure|Sci-Fi,...,1117,English,USA,PG-13,250000000,2015,21000,7.5,2.35,118000
9,Color,David Yates,375,153,282,10000,Daniel Radcliffe,25000,301956980,Adventure|Family|Fantasy|Mystery,...,973,English,UK,PG,250000000,2009,11000,7.5,2.35,10000
10,Color,Zack Snyder,673,183,0,2000,Lauren Cohan,15000,330249062,Action|Adventure|Sci-Fi,...,3018,English,USA,PG-13,250000000,2016,4000,6.9,2.35,197000


### Reading the records present

In [233]:
########### checking if node already been created or not ###############
def check_if_already_present():
    check_if_already_present = "select * from person"
    c = client.command(check_if_already_present)
    # print c
    d=[]

    for name in c:
        d.append([name.name, name.role, name.fblikes])

#     print d
    check_df = pd.DataFrame(list(d), columns=['name','role','fblikes'])
    return check_df

# # any(check_df.name == 'brad')

def check_if_already_present_movie():
    check_if_already_present = "select title , year from movie"
    c = client.command(check_if_already_present)
    # print c
    d=[]

    for name in c:
        d.append([name.title, name.year])

#     print d
    check_df_movie = pd.DataFrame(list(d), columns=['title','year'])
    return check_df_movie




## Inserting the records

In [234]:
# method to create the edge and the link when you have designed schema for your graphdb
def creating_records(imdb_df):
    for index, row in imdb_df.iterrows():
        ### creating records -- all three actors and director
        check_df = check_if_already_present()
        if(any(check_df.name == row["actor_1_name"])):
            print "node"+row["actor_1_name"] +" is already present!!"
        else:
            command_to_create_actor_1_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_1_name"]+'"' + ','+ str(row["actor_1_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            print(command_to_create_actor_1_node_class_person)
            client.command(command_to_create_actor_1_node_class_person)
        #############
        if(any(check_df.name == row["actor_2_name"])):
            print "node"+row["actor_2_name"] +" is already present!!"
        else:
            command_to_create_actor_2_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_2_name"]+'"' + ','+ str(row["actor_2_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            print(command_to_create_actor_2_node_class_person)
            client.command(command_to_create_actor_2_node_class_person)
        #############
        if(any(check_df.name == row["actor_3_name"])):
            print "node"+row["actor_3_name"] +" is already present!!"
        else:
            command_to_create_actor_3_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_3_name"]+'"' + ','+ str(row["actor_3_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            print(command_to_create_actor_3_node_class_person)
            client.command(command_to_create_actor_3_node_class_person)
        ############
        if(any(check_df.name == row["actor_1_name"])):
            print "node"+row["actor_1_name"] +" is already present!!"
        else:
            command_to_create_actor_4_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["director_name"]+'"' + ','+ str(row["director_facebook_likes"])+',' +'"' +'director'+'"' + ")"
            print(command_to_create_actor_4_node_class_person)
            client.command(command_to_create_actor_4_node_class_person)
        ################################ records created for the three actors , director ##############################################
         
        command_to_create_movie  = "INSERT INTO movie (title, year, durationInMins, imdbRating, genre, plotKeywords, numCriticForReviews, movieFacebookLikes) VALUES (" +'"' +row["movie_title"]+'"' + ','+ str(row["title_year"])+','+str(row["duration"])+','+str(row["imdb_score"])+',' +'"'+row["genres"]+'"' +','+'"' +row["plot_keywords"]+'"'+','+str(row["num_critic_for_reviews"])+','+str(row["movie_facebook_likes"])+")"
        print command_to_create_movie
        client.command(command_to_create_movie)





In [235]:
creating_records(imdb_df)

INSERT INTO person (name, fblikes, role) VALUES ("CCH Pounder",1000,"actor")
INSERT INTO person (name, fblikes, role) VALUES ("Joel David Moore",936.0,"actor")
INSERT INTO person (name, fblikes, role) VALUES ("Wes Studi",855.0,"actor")
INSERT INTO person (name, fblikes, role) VALUES ("James Cameron",0.0,"director")
INSERT INTO movie (title, year, durationInMins, imdbRating, genre, plotKeywords, numCriticForReviews, movieFacebookLikes) VALUES ("AvatarÂ",2009.0,178.0,7.9,"Action|Adventure|Fantasy|Sci-Fi","avatar|future|marine|native|paraplegic",723.0,33000)
INSERT INTO person (name, fblikes, role) VALUES ("Johnny Depp",40000,"actor")
INSERT INTO person (name, fblikes, role) VALUES ("Orlando Bloom",5000.0,"actor")
INSERT INTO person (name, fblikes, role) VALUES ("Jack Davenport",1000.0,"actor")
INSERT INTO person (name, fblikes, role) VALUES ("Gore Verbinski",563.0,"director")
INSERT INTO movie (title, year, durationInMins, imdbRating, genre, plotKeywords, numCriticForReviews, movieFacebo

In [236]:
## method to find the cluster of a class 
def find_the_Cluster_id_of_a_class (classname):
    find_the_Cluster_id_of_a_class = "SELECT defaultClusterId from (SELECT expand( classes ) FROM metadata:schema) where name = '" + classname + "'"
    print(find_the_Cluster_id_of_a_class)
    c = client.command(find_the_Cluster_id_of_a_class)
    for ids in c:
        return ids.defaultClusterId
    



In [237]:
data = {
        'person': {
                "name": "John", 
                "role": "director",
                "fblikes": 400000.0,
                "born_in" : 1980
                },
        }

for key, value in data.iteritems():
        cluster_id = find_the_Cluster_id_of_a_class(key)
        print cluster_id
#################################################################################
def creating_records_noschema(data):
        id = client.record_create(cluster_id, data)
        print "record succesfully created with " + str(id)


SELECT defaultClusterId from (SELECT expand( classes ) FROM metadata:schema) where name = 'person'
17


In [238]:
# uncomment to check the results !!
# check_df = check_if_already_present()
# print check_df
# check_df_movie = check_if_already_present_movie()
# print check_df_movie

                      name      role  fblikes
0              CCH Pounder     actor   1000.0
1              Johnny Depp     actor  40000.0
2          Christoph Waltz     actor  11000.0
3                Tom Hardy     actor  27000.0
4             Daryl Sabara     actor    640.0
5             J.K. Simmons     actor  24000.0
6             Brad Garrett     actor    799.0
7          Chris Hemsworth     actor  26000.0
8             Alan Rickman     actor  25000.0
9             Henry Cavill     actor  15000.0
10            Kevin Spacey     actor  18000.0
11      Giancarlo Giannini     actor    451.0
12           Tom Wilkinson     actor   1000.0
13    Pierfrancesco Favino     actor    216.0
14          Stephen Graham     actor   1000.0
15        Barry Sonnenfeld  director    188.0
16           Peter Jackson  director      0.0
17               Marc Webb  director    464.0
18            Ridley Scott  director      0.0
19             Chris Weitz  director    129.0
20           Peter Jackson  direct

In [239]:
def createRelationships():
      for index, row in imdb_df.iterrows():
            ############## create_edge_between_two_actors_1 and actor_2 ####################
            
            if((row["actor_1_name"]  in check_df.name.values) and (row["actor_2_name"]  in check_df.name.values)):
                command_to_create_edge_between_two_actors_1_and_2 = "create edge worked_with from (select from person where name = "+'"'+row["actor_1_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_2_name"]+'"'+")"
                print(command_to_create_edge_between_two_actors_1_and_2)
                client.command(command_to_create_edge_between_two_actors_1_and_2)
            else:
                   print "edge cant be created because vertext is not present !!!!!"
            
            
            ###### create_edge_between_two_actors_1 and actor_2 ###################
            if((row["actor_2_name"]  in check_df.name.values) and  (row["actor_3_name"]  in check_df.name.values) ):
                command_to_create_edge_between_two_actors_2_and_3 = "create edge worked_with from (select from person where name = "+'"'+row["actor_2_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_3_name"]+'"'+")"
                print(command_to_create_edge_between_two_actors_2_and_3)
                client.command(command_to_create_edge_between_two_actors_2_and_3)
            else:
                   print "edge cant be created because vertext is not present !!!!!"
               

            ##### create_edge_between_two_actors_1 and actor_2 ###################
            if((row["actor_2_name"]  in check_df.name.values) and ( row["actor_3_name"]  in check_df.name.values )):
                command_to_create_edge_between_two_actors_3_and_1 = "create edge worked_with from (select from person where name = "+'"'+row["actor_3_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_1_name"]+'"'+")"
                print(command_to_create_edge_between_two_actors_3_and_1)
                client.command(command_to_create_edge_between_two_actors_3_and_1)
            else:
                   print "edge cant be created because vertext is not present !!!!!"
                
            #### create_edge_between_two_actors_1 and actor_2 #####################
            if((row["actor_2_name"]  in check_df.name.values) and  (row["director_name"] in check_df.name.values) ):
                command_to_create_edge_between_actor_1_and_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_1_name"]+'"'+")"
                print(command_to_create_edge_between_actor_1_and_director)
                client.command(command_to_create_edge_between_actor_1_and_director)
            else:
                   print "edge cant be created because vertext is not present !!!!!"

            ##### create_edge_between_two_actors_1 and actor_2 #####################
            if((row["actor_2_name"]  in check_df.name.values) and ( row["director_name"] in check_df.name.values) ):
                command_to_create_edge_between_actor_2_and_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_2_name"]+'"'+")"
                print(command_to_create_edge_between_actor_2_and_director)
                client.command(command_to_create_edge_between_actor_2_and_director)
            else:
                   print "edge cant be created because vertext is not present !!!!!"
            
            ###### create_edge_between_two_actors_1 and actor_2 ######################
            if((row["actor_3_name"]  in check_df.name.values) and ( row["director_name"] in check_df.name.values) ):
                command_to_create_edge_between_actor_3_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_3_name"]+'"'+")"
                print(command_to_create_edge_between_actor_3_director)
                client.command(command_to_create_edge_between_actor_3_director)
            else:
                   print "edge cant be created because vertext is not present !!!!!"

            ###### movies and actors ###############################
            ####### create_edge_between_actors_1_and_movie #######
            if((row["actor_1_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values)):
                command_to_create_edge_between_actors_1_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_1_name"]+'"'+ ")"+" " +"to (select from movie where title = "+'"'+row["movie_title"]+'"'+")"
                print(command_to_create_edge_between_actors_1_and_movie)
                client.command(command_to_create_edge_between_actors_1_and_movie)
            else:
                   print "edge cant be created because vertext is not present !!!!!"

            ######create_edge_between_actors_2_and_movie
            if((row["actor_2_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values) ):
                command_to_create_edge_between_actors_2_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_2_name"]+'"'+ ")"+" " +"to (select from movie where title = "+'"'+row["movie_title"]+'"'+")"
                print( command_to_create_edge_between_actors_2_and_movie )
                client.command(command_to_create_edge_between_actors_2_and_movie)
            else:
                   print "edge cant be created because vertext is not present !!!!!"
            
            ###### create_edge_between_actors_1_and_movie 
            if((row["actor_3_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values)):
                command_to_create_edge_between_actors_3_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_3_name"]+'"'+ ")"+" " +"to (select from movie where title= "+'"'+row["movie_title"]+'"'+")"
                print(command_to_create_edge_between_actors_3_and_movie)
                client.command(command_to_create_edge_between_actors_3_and_movie)
            else:
                   print "edge cant be created because vertext is not present !!!!!"

        

In [240]:
createRelationships()
#create edge worked_in from (select from person where role = 'actor') to (select from person where role = 'actor')

create edge worked_with from (select from person where name = "CCH Pounder") to (select from person where name = "Joel David Moore")
create edge worked_with from (select from person where name = "Joel David Moore") to (select from person where name = "Wes Studi")
create edge worked_with from (select from person where name = "Wes Studi") to (select from person where name = "CCH Pounder")
create edge worked_with from (select from person where name = "James Cameron") to (select from person where name = "CCH Pounder")
create edge worked_with from (select from person where name = "James Cameron") to (select from person where name = "Joel David Moore")
create edge worked_with from (select from person where name = "James Cameron") to (select from person where name = "Wes Studi")
create edge acted_in from (select from person where name = "CCH Pounder") to (select from movie where title = "AvatarÂ")
create edge acted_in from (select from person where name = "Joel David Moore") to (select from m

### Most Mentioned
Movie with maximum no of facebook likes.


In [173]:
### query for the most mentioned 
def most_mentioned_movie():
    a = client.command(('select max(movieFacebookLikes) from movie '))

    for max_num in a :
        max_num = max_num.max
    print max_num
    most_mentioned_movie_object = client.command('select title from movie where movieFacebookLikes = ' + str(max_num))
    
    for titles in most_mentioned_movie_object :
        print titles.title

In [174]:
most_mentioned_movie()

349000
InterstellarÂ


### Clustering 

Cluster movies with rating above 7.



In [168]:
def movie_with_imdb_rating_above_7():
    title = []
    a = client.command(('select title from movie where imdbRating > 7 '))
    for titles in a :
        title.append(titles.title)
    title_df = pd.DataFrame(list(title), columns=['title'])    
    return title_df


In [170]:
movie_with_imdb_rating_above_7()